In [1]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
nasa_url = "https://mars.nasa.gov/news/"
twitter_url = "https://twitter.com/marswxreport?lang=en"
facts_url = "http://space-facts.com/mars/"
astro_base = 'https://astrogeology.usgs.gov'
astro_url = astro_base+"/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
url_list = [nasa_url,twitter_url,facts_url,astro_url]

## Here's the latest Mars Nasa News article

In [ ]:
driver = webdriver.Firefox()
driver.get(nasa_url)
driver.implicitly_wait(10)
nasa_soup = BeautifulSoup(driver.page_source,"lxml")
driver.close()

In [5]:
nasa_latest = {'title':nasa_soup.body.find('div',class_='content_title').text,
                'teaser':nasa_soup.body.find('div',class_='article_teaser_body').text}
nasa_latest

{'title': 'The MarCO Mission Comes to an End',
 'teaser': 'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'}

## Let's check twitter for the latest tweet from Mars Weather

In [14]:
twitter_r = r.get(twitter_url).text
twitter_soup = BeautifulSoup(twitter_r,'lxml')

In [15]:
twitter_soup.body.find('p',class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

'InSight sol 447 (2020-02-28) low -92.6ºC (-134.6ºF) high -11.1ºC (12.0ºF)\nwinds from the SSW at 5.8 m/s (13.1 mph) gusting to 20.2 m/s (45.2 mph)\npressure at 6.30 hPapic.twitter.com/fKd4VPHImB'

## Now for some Mars Facts

In [9]:
driver = webdriver.Firefox()
driver.get(facts_url)
driver.implicitly_wait(10)
facts_soup = BeautifulSoup(driver.page_source,"lxml")
driver.close()

In [13]:
facts_df = pd.read_html(str(facts_soup.body.find('table',id='tablepress-p-mars-no-2')))[0]
facts_df.columns = ['description','value']
facts_df = facts_df.set_index('description')
(facts_df)

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


## Last, but not least, Astrogeology has some cool images we want to save.

In [3]:
driver = webdriver.Firefox()
driver.get(astro_url)
driver.implicitly_wait(10)
astro_soup = BeautifulSoup(driver.page_source,"lxml")
driver.close()

We don't have the links to each of the images, so let's go get them as well as their Titles.

In [6]:
astro_d = [{'title':h.text,'url':astro_base+h.parent['href']} for h in astro_h]
astro_d

[{'title': 'Cerberus Hemisphere Enhanced',
  'url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'}]

Now we need to visit those links to find the image url.
I'll just go to the initial url and try to navigate from there.

This part is broken.

In [8]:
driver = webdriver.Firefox()
for h in astro_d:
    driver.get(h['url'])
    driver.implicitly_wait(10)
    soup = BeautifulSoup(driver.page_source,"lxml")
    img = soup.find('a',target='_blank')
    h.update({'img_url':img['href']})
driver.close()
astro_d

[{'title': 'Cerberus Hemisphere Enhanced',
  'url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]